In [21]:
from decimal import Decimal
import sys

# Mocking the classes based on file content
class Cash_Processor(object):
    def order_in_book_init_party(self, order_in_book):
        if order_in_book != None and order_in_book != []:
            order_in_book_val = order_in_book.get('price') * Decimal(order_in_book.get('quantity'))
            self.cash -= order_in_book_val
            self.cash_on_hold += order_in_book_val
        return 0

    def size_increase_cash_transfer(self, party, trade_val):
        if party == 'init_party':
            self.cash -= trade_val
        else:
            self.cash_on_hold -= trade_val
        return 0

    def size_decrease_cash_transfer(self, party, trade_val):
        if party == 'init_party':
            self.cash += trade_val
        else:
            self.cash += trade_val
            self.cash_on_hold -= trade_val
            self.cash += trade_val
        return 0

    def size_zero_cash_transfer(self, trade_val):
        self.cash += self.position_val - trade_val
        return 0

    def init_is_counter_cash_transfer(self, trade_val):
        self.cash_on_hold -= trade_val
        self.cash += trade_val
        return 0

    def modify_cash_transfer(self, qoute, order):
        # omitted for this test
        return 0

    def cancel_cash_transfer(self, order):
        # omitted for this test
        return 0

class Calculate(object):
    def cal_nav(self):
        self.nav =  self.cash + self.cash_on_hold + self.position_val
        return self.nav

    def cal_total_profit(self):
        self.total_profit = self.nav - self.init_nav
        return self.total_profit

    def cal_profit(self, position, mkt_val, raw_val):
        if position == 'long':
            self.profit = mkt_val - raw_val
        else:
            self.profit = raw_val - mkt_val
        return self.profit

    def mark_to_mkt(self, ID, mkt_price):
        price_diff = (self.VWAP - mkt_price, mkt_price - self.VWAP)[self.net_position >= 0]
        self.profit = Decimal(abs(self.net_position)) * price_diff
        raw_val = abs(self.net_position) * self.VWAP
        self.position_val = raw_val + self.profit
        self.prev_nav = self.nav
        self.cal_nav()
        self.cal_total_profit()
        return 0

class Account(Calculate, Cash_Processor):
    def __init__(self, ID, cash=0):
        self.ID = ID
        self.cash = Decimal(cash)
        self.cash_on_hold = Decimal(0)
        self.position_val = Decimal(0)
        self.init_nav = Decimal(cash)
        self.nav = Decimal(cash)
        self.prev_nav = Decimal(cash)
        self.net_position = 0
        self.VWAP = Decimal(0)
        self.profit = Decimal(0)
        self.total_profit = Decimal(0)
        self.num_trades = 0

    def _size_increase(self, trade, position, party, trade_val):
        total_size = abs(self.net_position) + Decimal(trade.get('quantity'))
        self.VWAP = (abs(self.net_position) * self.VWAP + trade_val) / total_size
        raw_val = total_size * self.VWAP
        mkt_val = total_size * trade.get('price')
        self.position_val = raw_val + self.cal_profit(position, mkt_val, raw_val)
        self.size_increase_cash_transfer(party, trade_val)
        return 0

    def _covered(self, trade, position):
        raw_val = abs(self.net_position) * self.VWAP
        mkt_val = abs(self.net_position) * trade.get('price')
        self.position_val = raw_val + self.cal_profit(position, mkt_val, raw_val)
        self.size_zero_cash_transfer(mkt_val)
        self.position_val = 0
        self.VWAP = 0
        return mkt_val

    def _size_decrease(self, trade, position, party, trade_val):
        size_left = abs(self.net_position) - Decimal(trade.get('quantity'))
        if size_left > 0:
            self.VWAP = (abs(self.net_position) * self.VWAP - trade_val) / size_left
            raw_val = size_left * self.VWAP
            mkt_val = size_left * trade.get('price')
            self.position_val = raw_val + self.cal_profit(position, mkt_val, raw_val)
        else:
            mkt_val = self._covered(trade, position)
        self.size_decrease_cash_transfer(party, trade_val)
        return 0

    def _covered_side_chg(self, trade, position, party):
        mkt_val = self._covered(trade, position)
        self.size_decrease_cash_transfer(party, mkt_val)
        new_size = Decimal(trade.get('quantity')) - abs(self.net_position)
        self.position_val = new_size * trade.get('price')
        self.VWAP = trade.get('price')
        self.size_increase_cash_transfer(party, self.position_val)
        return 0

    def _neutral(self, trade_val, trade, party):
        self.position_val += trade_val
        self.VWAP = trade.get('price')
        self.size_increase_cash_transfer(party, trade_val)

    def _net_long(self, trade_val, trade, party):
        if trade.get(party).get('side') == 'bid':
            self._size_increase(trade, 'long', party, trade_val)
        else:
            if self.net_position >= trade.get('quantity'):
                self._size_decrease(trade, 'long', party, trade_val)
            else:
                self._covered_side_chg(trade, 'long', party)

    def _net_short(self, trade_val, trade, party):
        if trade.get(party).get('side') == 'ask':
            self._size_increase(trade, 'short', party, trade_val)
        else:
            if abs(self.net_position) >= trade.get('quantity'):
                self._size_decrease(trade, 'short', party, trade_val)
            else:
                self._covered_side_chg(trade, 'short', party)

    def _update_net_position(self, side, trade_quantity):
        if self.net_position >= 0:
            if side == 'bid':
                self.net_position = Decimal(self.net_position) + Decimal(trade_quantity)
            else:
                self.net_position = Decimal(self.net_position) - Decimal(trade_quantity)
        else:
            if side == 'ask':
                self.net_position = Decimal(self.net_position) - Decimal(trade_quantity)
            else:
                self.net_position = Decimal(self.net_position) + Decimal(trade_quantity)
        return 0

    def process_acc(self, trade, party):
        self.num_trades += 1
        trade_val = Decimal(trade.get('quantity')) * trade.get('price')
        if self.net_position > 0:
            self._net_long(trade_val, trade, party)
        elif self.net_position < 0:
            self._net_short(trade_val, trade, party)
        else:
            self._neutral(trade_val, trade, party)
        self._update_net_position(trade.get(party).get('side'), trade.get('quantity'))
        self.cal_nav()
        return 0

# Test Script
acc = Account("TestTrader", 1000)
print(f"Initial: Cash {acc.cash}, NAV {acc.nav}")

# 1. Open Short 2 @ 100
trade1 = {
    'price': Decimal(100),
    'quantity': Decimal(2),
    'init_party': {'side': 'ask', 'ID':'TestTrader'}, # Selling
    'counter_party': {'ID': 'Other'}
}
print("\n--- Short 2 @ 100 ---")
acc.process_acc(trade1, 'init_party')
print(f"Cash {acc.cash}, PosVal {acc.position_val}, NAV {acc.nav}, VWAP {acc.VWAP}")
# Exp: Cash 800 (1000 - 200), PosVal 200 (Cost), NAV 1000

# 2. Mark to Market Price drop to 50
print("\n--- Mark to Market @ 50 ---")
acc.mark_to_mkt("TestTrader", Decimal(50))
print(f"Cash {acc.cash}, PosVal {acc.position_val}, NAV {acc.nav}, VWAP {acc.VWAP}")
# Exp: Cash 800, PosVal 300 (200 Cost + 100 Profit), NAV 1100

# 3. Partial Cover 1 @ 50
trade2 = {
    'price': Decimal(50),
    'quantity': Decimal(1),
    'init_party': {'side': 'bid', 'ID':'TestTrader'}, # Buying to cover
    'counter_party': {'ID': 'Other'}
}
print("\n--- Cover 1 @ 50 ---")
acc.process_acc(trade2, 'init_party')
print(f"Cash {acc.cash}, PosVal {acc.position_val}, NAV {acc.nav}, VWAP {acc.VWAP}")
# Exp (Based on analysis): Cash 850 (+50 trade val), PosVal 250 (Trapped Profit), NAV 1100

# 4. Final Cover 1 @ 50
trade3 = {
    'price': Decimal(50),
    'quantity': Decimal(1),
    'init_party': {'side': 'bid', 'ID':'TestTrader'}, 
    'counter_party': {'ID': 'Other'}
}
print("\n--- Cover 1 @ 50 (Final) ---")
acc.process_acc(trade3, 'init_party')
print(f"Cash {acc.cash}, PosVal {acc.position_val}, NAV {acc.nav}, VWAP {acc.VWAP}")
# Exp: Cash 1100, PosVal 0, NAV 1100


Initial: Cash 1000, NAV 1000

--- Short 2 @ 100 ---
Cash 800, PosVal 200, NAV 1000, VWAP 100

--- Mark to Market @ 50 ---
Cash 800, PosVal 300, NAV 1100, VWAP 100

--- Cover 1 @ 50 ---
Cash 850, PosVal 250, NAV 1100, VWAP 150

--- Cover 1 @ 50 (Final) ---
Cash 1100, PosVal 0, NAV 1100, VWAP 0


In [22]:
import sys
import os
from decimal import Decimal

if "../" not in sys.path:
    sys.path.append("../")
    
# Add current path to sys.path to ensure we can import the module
current_dir = os.getcwd()
sys.path.append(current_dir)

# Import the actual classes
from gym_continuousDoubleAuction.envs.account.account import Account

# Test Script
acc = Account("TestTrader", 1000)
print(f"Initial: Cash {acc.cash}, NAV {acc.nav}")

# 1. Open Short 2 @ 100
trade1 = {
    'price': Decimal(100),
    'quantity': Decimal(2),
    'init_party': {'side': 'ask', 'ID':'TestTrader'}, # Selling
    'counter_party': {'ID': 'Other'}
}
print("\n--- Short 2 @ 100 ---")
acc.process_acc(trade1, 'init_party')
print(f"Cash {acc.cash}, PosVal {acc.position_val}, NAV {acc.nav}, VWAP {acc.VWAP}")
# Exp: Cash 800 (1000 - 200), PosVal 200 (Cost), NAV 1000. VWAP 100.

# 2. Mark to Market Price drop to 50
print("\n--- Mark to Market @ 50 ---")
acc.mark_to_mkt("TestTrader", Decimal(50))
print(f"Cash {acc.cash}, PosVal {acc.position_val}, NAV {acc.nav}, VWAP {acc.VWAP}")
# Exp: Cash 800, PosVal 300 (200 Cost + 100 Profit? No. Short 2 @ 100 = 200 Liab. Price 50 = 100 Liab. Profit = 100).
# In this codebase: 'position_val' = |Net| * VWAP + Profit?
# mark_to_mkt:
# price_diff = (VWAP - mkt) * |Net| = (100 - 50) * 2 = 100.
# profit = 100.
# raw_val = |Net| * VWAP = 200.
# position_val = 200 + 100 = 300.
# NAV = 800 + 0 + 300 = 1100. 
# Matches previous run.

# 3. Partial Cover 1 @ 50
trade2 = {
    'price': Decimal(50),
    'quantity': Decimal(1),
    'init_party': {'side': 'bid', 'ID':'TestTrader'}, # Buying to cover
    'counter_party': {'ID': 'Other'}
}
print("\n--- Cover 1 @ 50 ---")
acc.process_acc(trade2, 'init_party')
print(f"Cash {acc.cash}, PosVal {acc.position_val}, NAV {acc.nav}, VWAP {acc.VWAP}")
# Exp (After Fix): 
# VWAP: Should remain 100.
# PnL on 1 unit: (100 - 50) = 50.
# Cash Flow: 2*Cost - Exit = 2*100 - 50 = 150.
# New Cash: 800 + 150 = 950.
# Remaining Pos: 1 Unit.
# Remaining PosVal: 1 * 50 = 50. (Or 1 * VWAP + Profit?). 
# _size_decrease calculates: raw_val (100) + cal_profit(1*50, 100) -> 50.
# So PosVal = 50.
# New NAV: Cash(950) + PosVal(50) = 1000? NO.
# Wait. NAV was 1100.
# If I start with 1000. Made 100 Profit (Unrealized). NAV 1100.
# Convert 50 Profit to Cash (Realized).
# Cash should include the 50 profit.
# New Cash = 950. (Includes 50 profit on 1 unit).
# Remaining unit: Unrealized Profit 50. Liab 50.
# PosVal should represent 1 unit marked to market?
# In this codebase, PosVal = |Net| * VWAP + Profit.
# Profit (from previous mark_to_mkt) was 100? No, profit is reset per tick/step?
# Account lines: "self.profit = Decimal(0)".
# But mark_to_mkt updates prev_nav.
# process_acc -> _size_decrease -> cal_profit.
# mkt_val = 50. raw_val = 100. profit = 100 - 50 = 50.
# PosVal = 100 + (-50? for short) -> Wait.
# cal_profit for short: raw_val (100) - mkt_val (50) = 50.
# PosVal = raw_val (100) + 50 = 150.
# So NAV = 950 + 150 = 1100.
# WAIT. PosVal should be the Asset Value?
# In this systems accounting:
# NAV = Cash + PosVal.
# If I am Short, NAV is Equity.
# So if Cash=950. Equity=1100. Then PosVal must be 150?
# But if I have a Liability of 50.
# And I have Cash 950.
# My Equity should be 900?
# No. Short Selling: Cash proceeds are held.
# Start: Cash 1000. Short 1 @ 100. Cash 900.
# System implies: Cash(900) + PosVal(200, 100 Collateral + 100 Proceeds?) NO.
# System implies PosVal holds the Collateral Value?
# _size_increase: raw_val = 100. mkt_val=100. Profit=0. PosVal=100.
# Cash 900 + PosVal 100 = 1000.
# So PosVal acts as "Collateral Posted" + "Unrealized PnL".
# If price drops to 50:
# raw=100. mkt=50. profit=50. PosVal = 150.
# Cash 900. Total 1050.
# Correct logic:
# I sold for 100. I owe share worth 50.
# My wealth = (1000 - 0) + (100 proceeds - 50 liability) = 1050.
# The system abstracts this as "PosVal".
# So:
# After covering 1:
# Cash 950.
# Remaining 1 unit Short. Entry 100. Mkt 50.
# raw=100. mkt=50. profit=50. PosVal=150.
# NAV = 950 + 150 = 1100.
# THIS MATCHES.
# AND: VWAP should be 100.

# 4. Final Cover 1 @ 50
trade3 = {
    'price': Decimal(50),
    'quantity': Decimal(1),
    'init_party': {'side': 'bid', 'ID':'TestTrader'}, 
    'counter_party': {'ID': 'Other'}
}
print("\n--- Cover 1 @ 50 (Final) ---")
acc.process_acc(trade3, 'init_party')
print(f"Cash {acc.cash}, PosVal {acc.position_val}, NAV {acc.nav}, VWAP {acc.VWAP}")
# Exp:
# Cash: 950 + 150 = 1100.
# PosVal: 0.
# NAV: 1100.
# VWAP: 0.



Initial: Cash 1000, NAV 1000

--- Short 2 @ 100 ---
Cash 800, PosVal 200, NAV 1000, VWAP 100

--- Mark to Market @ 50 ---
Cash 800, PosVal 300, NAV 1100, VWAP 100

--- Cover 1 @ 50 ---
Cash 950, PosVal 150, NAV 1100, VWAP 100

--- Cover 1 @ 50 (Final) ---
Cash 1100, PosVal 0, NAV 1100, VWAP 0


In [23]:
!pwd

/workspace/code/gym-continuousDoubleAuction_agent_based_0/gym_continuousDoubleAuction


In [24]:
!python3 -m gym_continuousDoubleAuction.test.test_account

/usr/bin/python3: Error while finding module specification for 'gym_continuousDoubleAuction.test.test_account' (ModuleNotFoundError: No module named 'gym_continuousDoubleAuction')


In [27]:
import unittest
import sys
import os
from decimal import Decimal

# if "../" not in sys.path:
#     sys.path.append("../")
# Ensure we can import the package
# Assuming running from root or this file overrides path issues if strictly relative imports in the implementation
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '../../../')))

# Mocking or importing dependent classes if needed, but here we just need Account
# from gym_continuousDoubleAuction.envs.account.account import Account
# We might need to handle the relative imports inside the package if running this script directly.
# The user's codebase uses `from .cash_processor import Cash_Processor` which requires this to be treated as a package.
# Better to assume running as `python -m gym_continuousDoubleAuction.envs.account.test_account` from root.

from gym_continuousDoubleAuction.envs.account.account import Account

class TestAccount(unittest.TestCase):
    def setUp(self):
        self.acc = Account(ID="TestUser", cash=1000)

    # --- Initialization Tests ---
    def test_initialization(self):
        self.assertEqual(self.acc.cash, Decimal(1000))
        self.assertEqual(self.acc.nav, Decimal(1000))
        self.assertEqual(self.acc.init_nav, Decimal(1000))
        self.assertEqual(self.acc.net_position, 0)
        self.assertEqual(self.acc.cash_on_hold, Decimal(0))
        self.assertEqual(self.acc.VWAP, Decimal(0))

    # --- Cash Processor Tests ---
    def test_order_in_book_init_party(self):
        """Test Cash_Processor: locking cash for new orders"""
        order = {'price': Decimal(100), 'quantity': Decimal(2)}
        self.acc.order_in_book_init_party(order)
        # Should lock 200 cash
        self.assertEqual(self.acc.cash, Decimal(800))
        self.assertEqual(self.acc.cash_on_hold, Decimal(200))

    def test_modify_cash_transfer_increase_size(self):
        """Test Cash_Processor: modify order to increase size/value"""
        # Original Order: 10 @ 10 = 100 value. Locked 100.
        # Modified Quote: 10 @ 15 = 150 value. Need to lock 50 more.
        
        # Setup: Manually lock some cash first to simulate state
        self.acc.cash = Decimal(900)
        self.acc.cash_on_hold = Decimal(100)
        
        old_order = type('obj', (object,), {'price': Decimal(10), 'quantity': Decimal(10)})
        new_quote = {'price': 15, 'quantity': 10}
        
        self.acc.modify_cash_transfer(new_quote, old_order)
        
        self.assertEqual(self.acc.cash_on_hold, Decimal(150)) # 100 + 50
        self.assertEqual(self.acc.cash, Decimal(850)) # 900 - 50

    def test_modify_cash_transfer_decrease_size(self):
        """Test Cash_Processor: modify order to decrease size/value"""
        # Setup: Locked 200.
        self.acc.cash = Decimal(800)
        self.acc.cash_on_hold = Decimal(200)
        
        old_order = type('obj', (object,), {'price': Decimal(20), 'quantity': Decimal(10)}) # 200 Val
        new_quote = {'price': 10, 'quantity': 10} # 100 Val
        
        self.acc.modify_cash_transfer(new_quote, old_order)
        
        self.assertEqual(self.acc.cash_on_hold, Decimal(100)) # 200 - 100
        self.assertEqual(self.acc.cash, Decimal(900)) # 800 + 100

    def test_cancel_cash_transfer(self):
        """Test Cash_Processor: cancel order"""
        self.acc.cash = Decimal(800)
        self.acc.cash_on_hold = Decimal(200)
        
        order = type('obj', (object,), {'price': Decimal(20), 'quantity': Decimal(10)})
        self.acc.cancel_cash_transfer(order)
        
        self.assertEqual(self.acc.cash_on_hold, Decimal(0))
        self.assertEqual(self.acc.cash, Decimal(1000))

    # --- Calculate Tests ---
    def test_mark_to_market_long(self):
        """Test Calculate: Mark to Market for Long position"""
        # Setup Long 10 @ 100
        self.acc.net_position = 10
        self.acc.VWAP = Decimal(100)
        # Position currently worthless? No, we need valid NAV/Cash setup?
        # Let's say we spent 1000 to buy these.
        self.acc.cash = Decimal(0)
        
        # Market Price 110. Profit 10 per share = 100 total.
        self.acc.mark_to_mkt("ID", Decimal(110))
        
        self.assertEqual(self.acc.profit, Decimal(100))
        # Position Value = Raw(1000) + Profit(100) = 1100
        self.assertEqual(self.acc.position_val, Decimal(1100))
        # NAV = Cash(0) + PosVal(1100) = 1100
        self.assertEqual(self.acc.nav, Decimal(1100))

    def test_mark_to_market_short(self):
        """Test Calculate: Mark to Market for Short position"""
        # Setup Short 10 @ 100.
        self.acc.net_position = -10
        self.acc.VWAP = Decimal(100)
        # We received proceeds? Short selling usually locks margin.
        # In this system, we verified cash goes down (100% margin).
        # So Cash should be 0 (if we started with 1000).
        self.acc.cash = Decimal(0)
        
        # Market Price 90. Profit 10 per share = 100 total.
        # Short Profit = (VWAP - Price) * Size = (100 - 90) * 10 = 100.
        self.acc.mark_to_mkt("ID", Decimal(90))
        
        self.assertEqual(self.acc.profit, Decimal(100))
        # Position Val = Raw(1000) + Profit(100) = 1100. (Includes Collateral + Profit)
        self.assertEqual(self.acc.position_val, Decimal(1100))
        self.assertEqual(self.acc.nav, Decimal(1100))

    # --- Account Core Logic Tests (The Fix) ---
    def test_long_buy_process(self):
        """Test process_acc for Buying logic"""
        # Buy 2 @ 100
        trade = {
            'quantity': Decimal(2),
            'price': Decimal(100),
            'init_party': {'side': 'bid', 'ID': 'TestUser'},
            'counter_party': {'ID': 'Other'}
        }
        self.acc.process_acc(trade, 'init_party')
        
        self.assertEqual(self.acc.cash, Decimal(800))
        self.assertEqual(self.acc.net_position, 2)
        self.assertEqual(self.acc.VWAP, Decimal(100))
        self.assertEqual(self.acc.nav, Decimal(1000))

    def test_short_sell_process(self):
        """Test process_acc for Short Selling logic"""
        # Sell 2 @ 100
        trade = {
            'quantity': Decimal(2),
            'price': Decimal(100),
            'init_party': {'side': 'ask', 'ID': 'TestUser'},
            'counter_party': {'ID': 'Other'}
        }
        self.acc.process_acc(trade, 'init_party')
        
        self.assertEqual(self.acc.cash, Decimal(800)) # 100% Margin
        self.assertEqual(self.acc.net_position, -2)
        self.assertEqual(self.acc.VWAP, Decimal(100))
        self.assertEqual(self.acc.nav, Decimal(1000))

    def test_partial_fill_short_cover_fix(self):
        """
        Verify the FIX: Cash Realization and Constant VWAP on Partial Short Cover
        """
        # 1. Short 2 @ 100
        trade_entry = {
            'quantity': Decimal(2),
            'price': Decimal(100),
            'init_party': {'side': 'ask', 'ID': 'TestUser'},
            'counter_party': {'ID': 'Other'}
        }
        self.acc.process_acc(trade_entry, 'init_party')
        
        # 2. Mark to Market @ 50
        self.acc.mark_to_mkt("TestUser", Decimal(50))
        self.assertEqual(self.acc.nav, Decimal(1100))
        
        # 3. Partial Cover 1 @ 50
        trade_cover = {
            'quantity': Decimal(1),
            'price': Decimal(50),
            'init_party': {'side': 'bid', 'ID': 'TestUser'},
            'counter_party': {'ID': 'Other'}
        }
        self.acc.process_acc(trade_cover, 'init_party')
        
        # VERIFICATIONS
        # 1. VWAP should still be 100 (Entry Price of remaining).
        self.assertEqual(self.acc.VWAP, Decimal(100))
        
        # 2. Cash Should Realize Profit.
        # Margin Released: 100. Profit: 50. Total Cash In: 150.
        # Start Cash 800 + 150 = 950.
        self.assertEqual(self.acc.cash, Decimal(950))
        
        # 3. Remaining Position Val
        # 1 unit @ 100 cost. Market 50. Profit 50. Val = 100 + 50 = 150.
        # Wait, Mark to MkT happened before.
        # In process_acc -> _size_decrease:
        # It calculates stats for remaining size.
        # raw_val = 1 * VWAP(100) = 100.
        # mkt_val = 1 * Price(50) = 50.
        # Profit = 100 - 50 = 50.
        # PosVal = 100 + 50 = 150.
        self.assertEqual(self.acc.position_val, Decimal(150))
        
        # 4. NAV Conserved
        # 950 + 150 = 1100.
        self.assertEqual(self.acc.nav, Decimal(1100))

    def test_full_cover_short(self):
        """Verify full loop completion"""
        # Short 1 @ 100
        self.acc.process_acc({
            'quantity': Decimal(1), 'price': Decimal(100),
            'init_party': {'side': 'ask', 'ID':'TestUser'}, 'counter_party': {'ID': 'O'}
        }, 'init_party')
        
        # Cover 1 @ 50
        self.acc.process_acc({
            'quantity': Decimal(1), 'price': Decimal(50),
            'init_party': {'side': 'bid', 'ID':'TestUser'}, 'counter_party': {'ID': 'O'}
        }, 'init_party')
        
        self.assertEqual(self.acc.net_position, 0)
        self.assertEqual(self.acc.VWAP, 0)
        self.assertEqual(self.acc.position_val, 0)
        self.assertEqual(self.acc.cash, Decimal(1050)) # 1000 + 50 Profit
        self.assertEqual(self.acc.nav, Decimal(1050))

    def test_partial_sell_long(self):
        """Verify Partial Sell of Long Position (Standard Logic)"""
        # Buy 2 @ 100
        self.acc.process_acc({
            'quantity': Decimal(2), 'price': Decimal(100),
            'init_party': {'side': 'bid', 'ID':'TestUser'}, 'counter_party': {'ID': 'O'}
        }, 'init_party')
        
        self.assertEqual(self.acc.cash, Decimal(800))
        
        # Sell 1 @ 150
        self.acc.process_acc({
            'quantity': Decimal(1), 'price': Decimal(150),
            'init_party': {'side': 'ask', 'ID':'TestUser'}, 'counter_party': {'ID': 'O'}
        }, 'init_party')
        
        # 1. Cash In: 150 (Trade Val).
        # New Cash: 800 + 150 = 950.
        self.assertEqual(self.acc.cash, Decimal(950))
        
        # 2. VWAP: 100.
        self.assertEqual(self.acc.VWAP, Decimal(100))
        
        # 3. Remaining Val: 1 @ 150.
        # Raw 100. Mkt 150. Profit 50. PosVal 150.
        self.assertEqual(self.acc.position_val, Decimal(150))
        
        # 4. NAV: 950 + 150 = 1100. (Profit 50).
        self.assertEqual(self.acc.nav, Decimal(1100))

if __name__ == '__main__':
    unittest.main()


NameError: name '__file__' is not defined